In [ ]:
import csv

dic = 'data/PUMSDataDict15.txt'
file = 'data/ss15pusa/ss15pusa.csv'

fields = []
with open(dic, 'r', encoding='mac_roman', newline='') as fi:
    index = 0
    line = fi.readline()
    flag = False
    prev = None
    while (line):
        if not flag and (line.find('PERSON RECORD') != -1):
            flag = True
            continue
        if flag and line[0:3].isupper() and line[0:2].find(' ') == -1 and prev=="\n":
            desc = fi.readline()
            print(index, line.split('\t')[0]+ " " + desc)
            index += 1
            fields.append(line.split('\t')[0]+ " " + desc)
        prev = line
        line = fi.readline()

In [ ]:
indeces = [4, 7, 8, 10, 11, 12, 13, 14, 18, 19, 20, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 37, 41, 
           42, 59, 62, 64, 66, 81, 84, 88, 91, 96, 97, 102, 103, 105, 110]
indeces = [index for index in indeces]
income = 101
race = 110
fields = [fields[i] for i in indeces]
for i in range(len(fields)):
    print(i, fields[i])

In [ ]:
with open(file, 'r', encoding='mac_roman') as fi, open('cleaned_acs_1.csv', 'w', encoding='mac_roman') as fo:
    reader = csv.reader(fi)
    writer = csv.writer(fo)
    count = 0
    
    for line in reader:
        # remove missing race and income values
        if (line[race] != '' and line[income] != ''):
            count += 1
            if (count % 25000 == 0):
                print(count)
            data = [line[i] for i in indeces]
            #recode missing as -1
            writer.writerow(['-1'if entry == '' else entry for entry in data])

In [ ]:
with open('cleaned_acs_1.csv', 'r', encoding='mac_roman') as fi:
    reader = csv.reader(fi)
    headers = next(reader)
    image = {i: set() for i in range(len(headers))}
    
    for line in reader:
        for i in range(len(headers)):
            image[i].add(line[i])
for i in range(len(headers)):
    print(i, headers[i], len(image[i]), list(image[i])[0:15])

In [ ]:
# bin continuous values - need to preprocess data for that 
age = 1
age_l = []
travelt = 18
travelt_l = []
pap = 25
pap_l = []
income = 35
income_l = []
race= 38

with open('cleaned_acs_1.csv', 'r', encoding='mac_roman') as fi:
    reader = csv.reader(fi)
    next(reader)
        
    for line in reader:
        age_l.append(int(line[age]))
        travelt_l.append(int(line[travelt]))
        pap_l.append(int(line[pap]))
        income_l.append(int(line[income]))

In [ ]:
import numpy as np
income_q = {1:np.quantile(income_l, 0.25), 2:np.quantile(income_l, 0.50), 
            3:np.quantile(income_l, 0.75), 4:np.quantile(income_l, 1), 5:np.quantile(income_l, 0.05)}
age_q = {1:np.quantile(age_l, 0.25), 2:np.quantile(age_l, 0.50), 
            3:np.quantile(age_l, 0.75), 4:np.quantile(age_l, 1)}
travelt_q = {1:np.quantile(travelt_l, 0.25), 2:np.quantile(travelt_l, 0.50), 
            3:np.quantile(travelt_l, 0.75), 4:np.quantile(travelt_l, 1)}
pap_q = {1:np.quantile(pap_l, 0.25), 2:np.quantile(pap_l, 0.50), 
            3:np.quantile(pap_l, 0.75), 4:np.quantile(pap_l, 1)}

In [ ]:
def get_q(index, quartiles, line):
    if int(line[index]) < quartiles[1]:
        line[index] = '0'
    elif int(line[index]) < quartiles[2]:
        line[index] = '1'
    elif int(line[index]) < quartiles[3]:
        line[index] = '2'
    else:
        line[index] = '3'
    return line

with open('cleaned_acs_1.csv', 'r', encoding='mac_roman') as fi, open('cleaned_acs_2.csv', 'w', encoding='mac_roman') as fo:
    reader = csv.reader(fi)
    writer = csv.writer(fo)
    
    # prepare the headers
    headers = next(reader)
    headers.append('race-binary')
    writer.writerow(headers)
    
    for line in reader:        
        # make race binary: 1 for white, 0 otherwise 
        line.append('1' if line[race] == '1' else '0')
        
        # bin income - 0 for low income and 1 for high
        # we're treating low income as bottom 50%
        if int(line[income]) < income_q[2]:
            line[income] = '0'
        else:
            line[income] = '1'
        
        # bin age by quartile: 0, 1, 2, 3
        line = get_q(age, age_q, line)
        
        # same for travel time and pap
        line = get_q(travelt, travelt_q, line)
        line = get_q(pap, pap_q, line)
        
        writer.writerow(line)